# Neural networks with PyTorch - Solutions

<hr style="clear:both">

This notebook is part of a series of exercises for the CIVIL-226 Introduction to Machine Learning for Engineers course at EPFL and adapted for the ME-390. Copyright (c) 2021 [VITA](https://www.epfl.ch/labs/vita/) lab at EPFL  
Use of this source code is governed by an MIT-style license that can be found in the LICENSE file or at https://www.opensource.org/licenses/MIT

**Author(s):** David Mizrahi

**Adapted by:** Sabri El Amrani
<hr style="clear:both">

In this exercise, we'll cover the basics of the [PyTorch](https://pytorch.org) package and use it to implement simple neural networks.

## Installation

In [ ]:
# Verify that PyTorch and torchvision are installed correctly
# Note: You may need to restart your kernel and re-run this cell before running the following cells
import torch

In [ ]:
torch.__version__

#### For Google Colab
You can run this notebook in Google Colab using the following link:
https://colab.research.google.com/github/SYCAMORE-Lab/ME390-2024/blob/main/Exercises/08-neural-nets/neural_nets_pytorch_Sol.ipynb

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
if IN_COLAB:
  # Clone the entire repo to access the files
  !git clone -l -s https://github.com/SYCAMORE-Lab/ME390-2024.git cloned-repo
  %cd cloned-repo/Exercises/08-neural-nets/

# Exercise 1: Deep Learning with PyTorch

PyTorch's website offers a tutorial on the basics of PyTorch, which can be found  at the following link: https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html

We recommend completing this tutorial before moving on to the second exercise.

# Exercise 2: Simple neural nets for classification with PyTorch

## 0. Introduction

In this exercise, we'll implement classifiers with PyTorch.
More specifically, we'll compare a simple logistic regression model to a simple neural network on the [Electrical Fault detection and classification](https://www.kaggle.com/datasets/esathyaprakash/electrical-fault-detection-and-classification) dataset.

*__Background:__ You're an electrical engineer working for a leading power systems research firm, specializing in transmission line protection and fault detection. Your team has compiled a dataset that includes measurements from various electrical power system transmission lines, documenting instances of faults and normal operating conditions. Your task is to develop a predictive model using artificial neural networks (ANNs) to detect and classify faults in transmission lines based on these measurements. This model will assist power system operators in maintaining system stability by enabling faster and more accurate fault detection, thereby minimizing power outages and enhancing the reliability of the power grid. The efficient operation of such a model is crucial for the protection of high-capacity generating power plants and the synchronized electrical power grids, ensuring the stability and robustness of the overall power system.*

<img src="images/electrical.jpg" style="width:500px"/>

[Source](https://www.usgs.gov/media/images/high-voltage-power-lines)

Here is the general pipeline used to train neural networks with PyTorch:

```
1. Load the dataset
2. Initialize a dataloader, define data transforms
3. Define and instantiate network architecture
4. Choose a loss function
5. Choose an optimizer
6. Define the training loop & number of epochs
7. Train the model
8. Check validation accuracy, visualize results, adjust hyper-parameters (not done in this exercise)
9. Repeat steps 2-9 until satisfied with validation accuracy (not done in this exercise)
10. Check test accuracy
```

*Recall, Epoch: number of times that the learning algorithm will work through the entire training set*

In this exercise, we'll follow these steps to implement our classifiers. Let's get started!

## 1. Imports

In [ ]:
# Pandas and scikit-learn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# Progress bar
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
import random
import numpy as np

# Helper files
import helpers
import metrics

Here is a brief description of these imported packages:

**PyTorch:**
- `torch.nn` Contains the basic building blocks to implement neural nets (incl. different types of layers and loss functions) | [Documentation](https://pytorch.org/docs/stable/nn.html)
- `torch.nn.functional` A functional (stateless) approach to torch.nn, often used for stateless objects (e.g. ReLU) | [Documentation](https://pytorch.org/docs/stable/nn.functional.html) | [More info](https://discuss.pytorch.org/t/what-is-the-difference-between-torch-nn-and-torch-nn-functional/33597/2)
- `torch.optim` A package implementing various optimization algorithms, such as SGD and Adam | [Documentation](https://pytorch.org/docs/stable/optim.html)

**`tqdm`:** Popular package used to show progress bars | [Documentation](https://tqdm.github.io/)

**`helpers`**: Contains functions to help visualize data and predictions

**`metrics`:** Contains two simple classes that help keep track and compute the loss and accuracy over a training epoch

## 2. Loading and visualizing the data

### 2.1. Dataset

Let's start by loading our dataset using pandas.

In [ ]:
# Load the CSV data
file_path = 'data/fault_classification.csv'
data = pd.read_csv(file_path)

# Split the features and target
X = data[['Ia', 'Ib', 'Ic', 'Va', 'Vb', 'Vc']]
y = data['Fault_Type']

# Encode categorical labels to integers
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)  # Use long tensor for classification labels
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create TensorDataset and DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
print(f"Samples in training data: {len(train_dataset)}")
print(f"Samples in test data: {len(test_dataset)}")

In [ ]:
# Show the mapping from target value to class name using label_encoder.classes_
class_names = label_encoder.classes_
class_name_mapping = {class_id: class_name for class_id, class_name in enumerate(class_names)}
print(class_name_mapping)

### 2.2. Dataloader

When training neural networks, we usually use mini-batches of data for each forward + backward pass. 

In order to obtain those mini-batches, we must pass our dataset through `torch.utils.DataLoader`, which combines the dataset and a sampler, and returns an iterable over the data and labels of our dataset.

In this exercise, we'll pick a batch size of 32.

In [ ]:
batch_size = 32

# Create TensorDataset and DataLoader for training data
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Create TensorDataset and DataLoader for test data
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

Let's take a look at one of our batches.

In [ ]:
features, target = next(iter(train_loader))

Let's check the size of a batch of our dataloader.

In [ ]:
print(features.shape)

In [ ]:
print(target.shape)

If everything looks good, let's build our first model using PyTorch.

## 3. Simple model

First, we'll build a simple classifier (akin to logistic regression).

###  3.1 Network architecture

#### Short primer on `nn.Module`

In PyTorch, each neural net architecture is a subclass of `nn.Module` ([Documentation](https://pytorch.org/docs/stable/generated/torch.nn.Module.html))

To quote an [official tutorial](https://pytorch.org/tutorials/beginner/nlp/deep_learning_tutorial.html?highlight=module):

> All network components should inherit from `nn.Module` and override the `forward()` method. That is about it, as far as the boilerplate is concerned. Inheriting from `nn.Module` provides functionality to your component. For example, it makes it keep track of its trainable parameters, you can swap it between CPU and GPU with the `.to(device)` method, where device can be a CPU device `torch.device("cpu")` or CUDA device `torch.device("cuda:0")`.


In order to implement your model, you'll therefore need to fill in two methods:

**`__init__()`**: 

- Initialize your layers here, so that the module can keep track of these layers' parameters. 
- It is not necessary to initialize layers with no learnable parameters (e.g. `ReLU`), as you can use  the`nn.functional` API for those if you want.

**`forward()`**:

- Define your model architecture here (i.e. call your layers in the desired order). You can use any of the Tensor operations in the `forward` function.
- This function defines the computation performed at every call. The backward function (where gradients are computed) is automatically defined for you using autograd. 

The learnable parameters of a model are returned by `model.parameters()`.

#### One layer neural net

You should implement a one-layer network as follows:
- A fully-connected layer from the input of shape (6,) to the output layer of shape (13,) as we have 13 different classes.

Note that there is no hidden layer here. 

Furthermore, the output layer provides a real value corresponding to each of the 6 classes. Hence, to use this network for classification, we can use a cross-entropy loss:: [`CrossEntropyLoss()` module](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html). Hence, we can obtain the probability of each of the 6 classes after training the network.

As a result, we also added a `predict()` method that adds this softmax layer, it'll be useful later on.

Notice that we use `class` from python to define the neural network. For a brief introduction to this, go back to Section 8 of the exercise 01-python.

**One layer neural net:**

<img src="images/1_layer_net.png" style="width:300px"/>

In [ ]:
class OneLayerNet(nn.Module):
    """1-Layer Fault Classifier"""

    def __init__(self, input_size, output_size):
        super().__init__()
        self.fc = nn.Linear(input_size, output_size)

    def forward(self, x):
        out = self.fc(x)
        return out

    def predict(self, x):
        logits = self.forward(x)
        return F.softmax(logits, dim=1)

input_size = 6
output_size = 13

model = OneLayerNet(input_size, output_size)

**Question:** How many trainable parameters (weights) does this network have?

**Answer:**
6 * 13 + 13 (bias) = 91 weights
    

### 3.2. Loss function and optimizer

As mentioned, we choose Cross Entropy loss as our loss function. For training, we use Stochastic Gradient Descent (SGD) with a learning rate of 0.02 as our optimizer.

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.02)

### 3.3. Training loop

The training loop can be defined as follows:

```
For each batch in the dataset:
   1. Load the batch
   2. Zero-out the accumulated gradients (PyTorch accumulates gradients during each iteration of an epoch. this is useful for some other neural network training tasks but not for our exercise).
   3. Run the forward pass through your model.
   4. Compute the loss.
   5. Run the backward pass, i.e. compute gradients of the loss w.r.t. to the weights.
   6. Update the weights using the optimizer.
```

Take a look at the `train()` function written for you below.

In [ ]:
def train(model, train_loader, loss_fn, optimizer, epochs, seed=None):
    """
    Train the model with the given parameters.
    
    Parameters:
    model (torch.nn.Module): The model to train.
    train_loader (torch.utils.data.DataLoader): DataLoader for the training data.
    loss_fn (torch.nn.Module): The loss function.
    optimizer (torch.optim.Optimizer): The optimizer.
    epochs (int): Number of epochs to train for.
    seed (int, optional): Random seed for reproducibility. Default is None.
    """

    if seed is not None:
        helpers.set_seed(seed)
    
    # Initialize metrics for loss and accuracy (assuming they are custom implementations)
    loss_metric = metrics.LossMetric()  # Placeholder for your LossMetric class
    acc_metric = metrics.AccuracyMetric(k=1)  # Placeholder for your AccuracyMetric class
    
    # Lists to store loss and accuracy for each epoch
    train_losses = []
    train_accuracies = []
    
    # Set the model in training mode
    model.train()
    
    for epoch in range(1, epochs + 1):
        # Iterate through the data
        for data, target in train_loader:
            # Zero out the gradients
            optimizer.zero_grad()
            
            # Forward pass
            output = model(data)
            
            # Compute the loss
            loss = loss_fn(output, target)
            
            # Backward pass
            loss.backward()
            
            # Optimizer step
            optimizer.step()
            
            # Update metrics
            loss_metric.update(loss.item(), data.size(0))
            acc_metric.update(output, target)
        
        # Compute and store metrics for the epoch
        epoch_loss = loss_metric.compute()
        epoch_acc = acc_metric.compute() * 100
        train_losses.append(epoch_loss)
        train_accuracies.append(epoch_acc)
        
        # Print epoch summary every 20 epochs
        if epoch % 20 == 0 or epoch == epochs:
            print(f"Epoch {epoch}/{epochs} - Train loss: {epoch_loss:.3f} | Train acc: {epoch_acc:.2f}%")
        
        # Reset metrics for next epoch
        loss_metric.reset()
        acc_metric.reset()
    
    # Plot the evolution of loss and accuracy
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(range(1, epochs + 1), train_losses, label='Train Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training Loss Over Epochs')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(range(1, epochs + 1), train_accuracies, label='Train Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.title('Training Accuracy Over Epochs')
    plt.legend()

    plt.tight_layout()
    plt.show()

**Question:** How many iterations are in each epoch? 

**Answer:** There are 6288/32 $\approx$ 175 iterations

Now that the train function is defined, it's time to actually train your model.

In [ ]:
# Train model for 50 epochs
train(model, train_loader, loss_fn, optimizer, epochs=50, seed=42)

### 3.4. Test accuracy

You should now have a model with approximately 35% training accuracy, but this doesn't tell the whole story. In order to actually estimate how good (or rather how bad in this case) our model is, we need to check its accuracy on the test set.

Look through the function `test()` which computes the test accuracy of a given model on a specific dataset.

This function iterates through a dataset once (using a DataLoader) and displays the accuracy.

In [ ]:
def test(model: torch.nn.Module, dataloader: torch.utils.data.DataLoader, seed=None):
    """
    Evaluates the accuracy of a neural network model on the given data.

    This function evaluates the performance of a neural network model by calculating
    the accuracy on the provided dataloader. It uses a progress bar to indicate
    the completion status during evaluation and prints the final accuracy.

    Args:
        model (torch.nn.Module): The neural network model to evaluate.
        dataloader (torch.utils.data.DataLoader): DataLoader for the evaluation data.

    Returns:
        None
    """
    if seed:
        helpers.set_seed(seed)

    # Initialize accuracy metric
    acc_metric = metrics.AccuracyMetric(k=1)

    # Progress bar setup
    pbar = tqdm(total=len(dataloader), leave=True)

    # Set the model in evaluation mode
    model.eval()

    with torch.no_grad():
        # Iterate through data
        for data, target in dataloader:
            # Forward pass
            out = model(data)

            # Update accuracy metric
            acc_metric.update(out, target)

            # Update progress bar
            pbar.update()

    # Compute and display accuracy
    test_acc = acc_metric.compute() * 100
    pbar.set_postfix_str(f"Acc: {test_acc:.2f}%")
    print(f"Accuracy: {test_acc:.2f}%")

In [ ]:
test(model, test_loader, seed=42)

**Expected result:** $\approx$ 36% test accuracy.

__Question:__ How does the test accuracy compare with the training accuracy? Do we have overfit?

__Answer:__ Training and test accuracies are very similar. There is no overfit. It is quite the opposite in fact, the model underfits: the logistic regression model is probably too simple for the data.

__Question:__ How does our logistic model compare with a random classifier?

__Answer:__ A random classifier would only get an accuracy of ~16.7% (as there are 6 classes). 35 % is therefore reassuringly already a good improvement.

Let's not stop there, though, and try to do better with just a few small changes.

## 4. Simple neural net (with hidden layers)

Time to implement (actual) neural networks. As you'll soon see, the entire pipeline is very similar, only the network architecture changes.

#### Network architecture

You should implement a three-layer fully-connected neural net (2 hidden layers) composed of the following:
- A fully-connected layer from the input of shape (6,) to the first hidden layer of shape (10, ), with ReLU as an activation.
- A fully-connected layer from the first hidden layer of shape (10,) to the second hidden layer of shape (10, ), with ReLU as an activation.
- A fully-connected layer from the second hidden layer of shape (10,) to the output layer of shape (13,).

Fully connected neural networks (FCNNs) are sometimes also referred to as multilayer perceptrons (MLP) in ML literature.

**Note:** You can use the OneLayerNet function above and simply fill in additional information for the layers below

**Three-layer neural net:**

<img src="images/3_layer_net.png" style="width:300px"/>

In [ ]:
class ThreeLayerNet(nn.Module):
    """3-Layer neural net"""
    
    def __init__(self, input_size, output_size, hidden_layer_size=10):
        super().__init__()

        self.fc1 = nn.Linear(input_size, hidden_layer_size)
        ### START CODE HERE ###
        ### Hint: you need two more linear layers here of appropriate dimension
        self.fc2 = nn.Linear(hidden_layer_size, hidden_layer_size)
        self.fc3 = nn.Linear(hidden_layer_size, output_size)
        ### END CODE HERE ###

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = F.relu(self.fc1(x))
        ### START CODE HERE ###
        ### Hint: you need one more relu layer and the output layer     
        x = F.relu(self.fc2(x))
        out = self.fc3(x)
        return out
        ### END CODE HERE ###

    def predict(self, x: torch.Tensor) -> torch.Tensor:
        """Predicts classes by calculating the softmax"""
        logits = self.forward(x)
        return F.softmax(logits, dim=1)


input_size = 6
output_size = 13

model = ThreeLayerNet(input_size, output_size)

**Question:** How many trainable parameters (weights) does this network have?

**Answer:**
- First layer: (6 * 10) + 10 (bias) = 70
- Second layer: (10 * 10) + 10 (bias) = 110
- Third layer: (10 * 13) + 13 (bias) = 143
- **Total: 70 + 110 + 143 = 323 weights**
    

#### Loss & optimizer

Use the same loss function and optimizer as in the previous section. Namely, the Cross-Entropy loss and SGD with lr=0.02.

In [ ]:
### START CODE HERE ###
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.02)
### END CODE HERE ###

#### Training

Use `train()` to launch the training process, use the same number of epochs and seed as in the previous part.

In [ ]:
### START CODE HERE ###
train(model, train_loader, loss_fn, optimizer, epochs=50, seed=42)
### END CODE HERE ###

#### Test accuracy

Compute the test accuracy using `test()` with a seed of 42.

In [ ]:
### START CODE HERE ###
test(model, test_loader, seed=42)
### END CODE HERE ###

**Expected result:** $\approx$ 83 % test accuracy.

Great job so far! Now that we know how to train a neural network for classification, let's implement a network for regression. You will see, the pipeline will hardly change in the process.

# Advanced (optional): Simple neural net for regression

### 0. Introduction

*__Background:__ You're a materials scientist working for a leading research institute focused on the electric breakdown behavior of materials. Your team has compiled a dataset that includes measurements of various semiconductor and insulator materials, documenting their intrinsic dielectric breakdown fields alongside several experimental and calculated features. Your task is to develop a predictive model that can estimate the intrinsic dielectric breakdown field of a material based on its properties. This model will assist in the discovery of new materials with superior dielectric properties and contribute to the formulation of phenomenological laws governing electric breakdown field strength. By understanding and predicting dielectric breakdown, you aim to enhance the reliability and efficiency of electronic devices and systems that rely on these materials.*

<img src="images/dielectric_breakdown.png" style="width:500px"/>

[Source](https://www.matsusada.com/application/ps/dielectric_breakdown_testing/)

### 1. Data loading & pre-processing

As usual, let's start by loading and pre-processing the data. The data for this exercise can be found [here](https://www.kaggle.com/datasets/chaozhuang/dielectric-breakdown-prediction-dataset/data). It is taken from the paper [From Organized High-Throughput Data to Phenomenological
Theory using Machine Learning: The Example of Dielectric
Breakdown](https://ramprasad.mse.gatech.edu/wp-content/uploads/2018/03/138.pdf).

In [ ]:
# Load data
df = pd.read_csv("data/dielectric.csv")
df = df.drop_duplicates()
df.head()

In [ ]:
X = df.drop(['name', 'log_breakdown_field', 'structure', 'category'], axis = 1).values
y = df[['log_breakdown_field']].values

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Normalize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

In [ ]:
X_train.shape

In [ ]:
batch_size = 4

# Create TensorDataset and DataLoader for training data
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Create TensorDataset and DataLoader for test data
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
print(f"Samples in training data: {len(train_dataset)}")
print(f"Samples in test data: {len(test_dataset)}")

In [ ]:
features, target = next(iter(train_loader))

In [ ]:
print(features.shape)

In [ ]:
print(target.shape)

### 2. Simple model for regression

#### Network architecture

You should implement a three-layer fully-connected neural net (2 hidden layers) composed of the following:
- A fully-connected layer from the input of shape (8,) to the first hidden layer of shape (8,), with ReLU as an activation.
- A fully-connected layer from the first hidden layer of shape (8,) to the second hidden layer of shape (8, ), with ReLU as an activation.
- A fully-connected layer from the second hidden layer of shape (8,) to the output layer of shape (1,).

As you can notice, network-wise, the main difference between classification and regression is the shape of the output. Rather than an output size equal to the number of classes, we have one node for the (one) real value that needs to be predicted. 

Notice also, that the softmax is not applied to the output during prediction. Indeed, we don't need a softmax that squeezes outputs between 0 and 1 anymore, but rather an output that can take any real value.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F 

class RegressionNet(nn.Module):
    """3-Layer neural net"""
    
    def __init__(self, input_size, output_size, hidden_layer_size=8):
        super().__init__()
        ### START CODE HERE ###
        ### Hint: you need three linear layers here of appropriate dimension
        self.fc1 = nn.Linear(input_size, hidden_layer_size)
        self.fc2 = nn.Linear(hidden_layer_size, hidden_layer_size)
        self.fc3 = nn.Linear(hidden_layer_size, output_size)
        ### END CODE HERE ###

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        ### START CODE HERE ###
        ### Hint: you need two relu layers and the output layer 
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        out = self.fc3(x)  # Linear output for regression
        return out
        ### END CODE HERE ###

    def predict(self, x: torch.Tensor) -> torch.Tensor:
        """Predicts excitation current for regression task"""
        prediction = self.forward(x)
        return prediction  # Return raw output for regression

Take a look at the training code below now. You might notice that it is almost identical to the code used for training a classification network. The main difference is the tracked metric. Instead of plotting the accuracy of the model, we will plot the [coefficient of determination $R^2$](https://en.wikipedia.org/wiki/Coefficient_of_determination) to assess the quality of our predictions.

In [ ]:
def train(model, train_loader, loss_fn, optimizer, epochs, seed=None):   
    """
    Trains a neural network model.

    This function trains a neural network model using the provided data loader, loss function, and optimizer.
    It tracks and plots the training loss and R² metrics over the specified number of epochs.

    Args:
        model (nn.Module): The neural network model to train.
        train_loader (DataLoader): DataLoader for the training data.
        loss_fn (callable): The loss function.
        optimizer (torch.optim.Optimizer): The optimizer to use for training.
        epochs (int): The number of epochs to train the model.
        seed (int, optional): Random seed for reproducibility. Default is None.

    Returns:
        None
    """
    
    if seed:
        helpers.set_seed(seed)
    
    # Initialize metrics for loss and R²
    loss_metric = metrics.LossMetric()  # Placeholder for your LossMetric class
    r2_metric = metrics.R2Metric()  # Placeholder for your R2Metric class
    
    # Lists to store loss and R² values
    train_losses = []
    train_r2 = []
    
    # Set the model in training mode
    model.train()
    
    for epoch in range(1, epochs + 1):
        # Iterate through the data
        for data, target in train_loader:
            # Zero out the gradients
            optimizer.zero_grad()
            
            # Forward pass
            output = model(data)
            
            # Compute the loss
            loss = loss_fn(output, target)
            
            # Backward pass
            loss.backward()
            
            # Optimizer step
            optimizer.step()
            
            # Update metrics
            loss_metric.update(loss.item(), data.size(0))
            r2_metric.update(output, target)
        
        # Compute average metrics for the epoch
        epoch_loss = loss_metric.compute()
        epoch_r2 = r2_metric.compute()
        train_losses.append(epoch_loss)
        train_r2.append(epoch_r2)
        
        # Print epoch summary every 10 epochs
        if epoch == 1 or epoch % 10 == 0 or epoch == epochs:
            print(f"Epoch {epoch}/{epochs} - Train loss: {epoch_loss:.3f} - R²: {epoch_r2:.3f}")
        
        # Reset metrics for next epoch
        loss_metric.reset()
        r2_metric.reset()
    
    # Plot the loss and R² values
    plt.figure(figsize=(12, 5))

    # Plotting loss
    plt.subplot(1, 2, 1)
    plt.plot(range(1, epochs + 1), train_losses, label='Train Loss')
    plt.xlabel('Epoch')
    plt.xscale('log')
    plt.ylabel('Loss')
    plt.title('Training Loss over Epochs')
    plt.legend()
    plt.grid(True)

    # Plotting R²
    plt.subplot(1, 2, 2)
    plt.plot(range(1, epochs + 1), train_r2, label='Train R²')
    plt.xlabel('Epoch')
    plt.xscale('log')
    plt.ylabel('R²')
    plt.title('Training R² over Epochs')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

Let's run the training now. Notice how we also change the loss used: rather than a cross-entropy loss, we use the Mean Squared Error, similar to what we did in the linear regression notebook.

We also opted for [Adam](https://www.geeksforgeeks.org/adam-optimizer/), a more performant optimizer than the usual SGD.

In [ ]:
input_size = 8
output_size = 1

### START CODE HERE ###
# Initialize the model
model = RegressionNet(input_size, output_size)

# Initialize the loss function, and the optimizer with a learning rate of 0.01 and a weight_decay of 0.4
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=0.4) 

# Train the model, for 1000 epochs and with a seed of 42
train(model, train_loader, loss_fn, optimizer, epochs=1000, seed=42)
### END CODE HERE ###

Now comes the time to test our trained model.

In [ ]:
def test(model: torch.nn.Module, dataloader: torch.utils.data.DataLoader, seed=None):
    """
    Evaluates a neural network model.

    This function evaluates a neural network model using the provided dataloader.
    It tracks and prints the Mean Absolute Error (MAE) and R² metrics.

    Args:
        model (torch.nn.Module): The neural network model to evaluate.
        dataloader (torch.utils.data.DataLoader): DataLoader for the evaluation data.

    Returns:
        None
    """
    if seed:
        helpers.set_seed(seed)

    # Initialize MAE and R² metrics
    mae_metric = metrics.MAEMetric()
    r2_metric = metrics.R2Metric()

    # Progress bar set-up
    pbar = tqdm(total=len(dataloader), leave=True)

    # Set the model in evaluation mode
    model.eval()

    with torch.no_grad():
        # Iterate through data
        for data, target in dataloader:
            # Forward pass
            out = model(data)

            # Update MAE and R² metrics
            mae_metric.update(out, target)
            r2_metric.update(out, target)

            # Update progress bar
            pbar.update()

    # End of evaluation, show MAE and R²
    test_mae = mae_metric.compute()
    test_r2 = r2_metric.compute()
    pbar.set_postfix_str(f"MAE: {test_mae:.4f}, R²: {test_r2:.4f}")
    print(f"Test MAE: {test_mae:.4f}, Test R²: {test_r2:.4f}")

    # Reset metrics
    mae_metric.reset()
    r2_metric.reset()

In [ ]:
### START CODE HERE ###
# Test the model with a seed of 42
test(model, test_loader, seed=42)
### END CODE HERE ###

You should obtain a [Mean Absolute Error](https://en.wikipedia.org/wiki/Mean_absolute_error) of approximately 0.43, and an $R^2$ of roughly 0.68.

Not bad right? It roughly means that almost 70% of the variation of the data can be explained by our model. It's probably not the best performance that can be obtained on this data, but further tuning and longer trainings would be needed for that.

Congratulations on training your first neural networks! In the next session, you will learn to train a special type of network: convolutional neural networks.

## (Optional) Additional PyTorch resources
- PyTorch cheat sheet: https://pytorch.org/tutorials/beginner/ptcheat.html
- Other PyTorch tutorials: https://pytorch.org/tutorials/index.html
- PyTorch recipes: https://pytorch.org/tutorials/recipes/recipes_index.html (bite-sized code examples on specific PyTorch features)
- PyTorch examples: https://github.com/pytorch/examples